<a href="https://colab.research.google.com/github/natatsypora/plotly_charts/blob/main/Waffle_chart_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np

## Get and read Data

In [ ]:
# get the file from Google Drive
! gdown -- 1U9GNdGpxm0rEo7WmE3Y9dKoDVlr7n5ye

In [5]:
# read data
df_vs_last = pd.read_excel('/content/teunot_by_month_type2022.xlsx',
                           sheet_name='yoy', index_col=0)
df_vs_last

,Total_2022,Slight_2022,Serious_2022,Fatal_2022,Total_2021,Slight_2021,Serious_2021,Fatal_2021
Month,,,,,,,,
January,881,698,160,23,862,697,149,16
February,826,648,150,28,916,749,138,29
March,957,744,185,28,1093,867,201,25
April,927,684,210,33,1046,809,195,42
May,985,694,261,30,935,734,173,28
June,892,657,212,23,1077,838,207,32
July,915,686,211,18,978,752,201,25
August,845,630,189,26,982,756,199,27
September,824,614,186,24,848,638,183,27


In [4]:
# calculate total by columns
df_vs_last.sum(axis=0).to_frame().T

,Total_2022,Slight_2022,Serious_2022,Fatal_2022,Total_2021,Slight_2021,Serious_2021,Fatal_2021
0,10404,7765,2320,319,11554,9010,2208,336


In [6]:
# choose three columns for chart and calculate percenage of total
data_for_waffle = (df_vs_last.sum(axis=0).to_frame().T.iloc[:, 1:4]/10_404).T
data_for_waffle

,0
Slight_2022,0.746348
Serious_2022,0.222991
Fatal_2022,0.030661


## Create Waffle Chart using go.Heatmap

In [106]:
# define colors for waffle charts
choose_color = ['#88b37c','#4085bb', '#FF0000']

In [116]:
# @title heatmap_waffle_subplots function
def heatmap_waffle_subplots(df_percentage, title=None):
    """ Creates a heatmap waffle chart with subplots.
    Parameters:
        df_percentage (DataFrame): A DataFrame containing the percentage data for the waffle chart.
        title (str, optional): The title of the chart. Defaults to None.
    Returns:
        fig (Figure): A Plotly figure object representing the heatmap waffle chart with subplots.
    """
    fig = make_subplots(rows=1, cols=3,
                        subplot_titles=[f'<b>{t[0].split("_")[0]}</b>' for t in df_percentage.itertuples()],
                        horizontal_spacing=0.03,
                        specs=[[{"type": "heatmap"}]*3,])

    # Make array for a waffle chart 10x10
    arr = [i for i in range(1, 100+1)]
    z_values = np.reshape(arr, (10, 10))[::-1]

    for i, p in enumerate(df_percentage.iloc[:, 0].values):
        # Define colors for each heatmap
        fill_color = [[0.0, choose_color[i]], [round(p,2), choose_color[i]], [round(p,2), 'lightgrey'], [1.0, 'lightgrey']]
        # Create heatmaps
        fig.add_trace(go.Heatmap(hoverinfo='skip',
                                 z=z_values,
                                 xgap=2, ygap=2,
                                 colorscale=fill_color,
                                 showscale=False
                     ), row=1, col=i+1)
        # Add annotation for each heatmap
        fig.add_annotation(text = f'<b>{p:.0%}</b>',
                           font_color=choose_color[i],
                           showarrow=False,
                           x = 0.155+i/2.9,
                           y = -0.15,
                           xref='paper',
                           yref='paper' ,
                           xanchor='center')

    # Define layout and axis properties
    fig.update_layout(title=title, title_x=0.5, title_y=0.95,
                      font_color='rgba(0, 0, 0, 0.5)', font_size=18,
                      plot_bgcolor='rgba(255, 255, 255, 0.5)',
                      paper_bgcolor='#F7F7F7',
                      width=800, height=350,
                      margin=dict(l=30, r=30, b=50, t=80))
    fig.update_xaxes(visible=False)
    fig.update_yaxes(autorange="reversed", visible=False)

    return fig

hm = heatmap_waffle_subplots(data_for_waffle, '<b>Road Accidents 2022 by Severity<b>')
hm.show(config = {'displayModeBar': False})

In [109]:
# view annotation for first subtitle
hm.layout['annotations'][0]

layout.Annotation({
    'font': {'size': 16},
    'showarrow': False,
    'text': '<b>Slight</b>',
    'x': 0.15666666666666665,
    'xanchor': 'center',
    'xref': 'paper',
    'y': 1.0,
    'yanchor': 'bottom',
    'yref': 'paper'
})

In [117]:
# update font color, font size and positions for subtitles
for i in range(3):
    hm.layout['annotations'][i]['font_color'] = choose_color[i]
    hm.layout['annotations'][i]['font_size'] = 18
    hm.layout['annotations'][i]['y'] = 1.03

hm.show()

## Create Waffle Chart using go.Table

In [58]:
# define colors for waffle charts
table_colors = ['#88b37c','#4085bb', '#FF0000',
                'rgba(0, 150, 0, 0.1)', 'rgba(0, 0, 150, 0.1)', 'rgba(150, 0, 0, 0.1)']

In [113]:
# @title table_waffle_chart_subplots function
def table_waffle_chart_subplots(df_percentage, title):
    """ Creates a table waffle chart with subplots.
    Parameters:
        df_percentage (DataFrame): A DataFrame containing the percentage data for the waffle chart.
        title (str, optional): The title of the chart. Defaults to None.
    Returns:
        fig (Figure): A Plotly figure object representing the table waffle chart with subplots.
    """
    fig = make_subplots(rows=1, cols=3,
                        subplot_titles=[f'<b>{t[0].split("_")[0]} ({t[1]:.0%})</b>' for t in df_percentage.itertuples()],
                        horizontal_spacing=0.03,
                        specs=[[{"type": "table"}]*3,])

    # Make array for a waffle chart 10x10
    arr = [i for i in range(1, 100+1)]
    cell_values = np.reshape(arr, (-1, 10))[::-1].transpose()

    for i, p in enumerate(df_percentage.iloc[:, 0].values):
        # Define colors for each table
        cell_colors = [[table_colors[i] if el <= round(p*100, 0) else table_colors[i+3] for el in row]  for row in cell_values]

        # Create tables
        fig.add_trace(go.Table(header=dict(values=['',], height=0),   # empty header with zero height
                               cells=dict(values=cell_values,
                               height=22,
                               font_color='rgba(255, 255, 255, 0.0)', # transparent color for text
                               fill_color=cell_colors)
                              ), row=1, col=i+1)

    # Define layout and axis properties
    fig.update_layout(title=title, title_x=0.5, title_y=0.95,
                      font_color='rgba(0, 50, 100, 0.7)',
                      paper_bgcolor='#F7F7F7',
                      font_size=18,
                      width=800, height=350,
                      margin=dict(l=30, r=30, b=30, t=100))

    # Update font color, font size and positions for subtitles
    for i in range(3):
        fig.layout['annotations'][i]['font_color'] = table_colors[i]
        fig.layout['annotations'][i]['font_size'] = 18
        fig.layout['annotations'][i]['y'] = 1.05

    return fig

hm2 = table_waffle_chart_subplots(data_for_waffle, 'Road Accidents 2022 by Severity')
hm2.show()